This file demonstrates the theoretical performance of the $B$ and $\lambda$ recovery procedures. It does so by creating something called *b_measurements*, which computes the theoretical expected measurement for some paramaterization (see *Fair_QME_Parameterization* in *fpme_utils.py*).

Let $ng$ := number of groups.

Explanation:

A paremeterization is (typically) a tuple (a, b) that specifies which TWO groups should get rate $s$. All other groups should get the trivial rate. As shown in the paper, this results in the metric becoming a sum of SOME (not all) $B$ matrices. Specifically, it would be all $B_{ij}$ such that $ i = a \text{ or } b $ and $j \neq a, b $. This sum is multiplied by $ \frac{\lambda}{1 - \lambda} $. Thus our "measurement" is:

$$ M_{ab} = \frac{\lambda}{1 - \lambda} * B^m_{ab} $$
$$ B^m_{ab} = \sum_{i \in \{a,b\}, j \neq a,b} B_{ij} $$

*fpme.ipynb* shows how this can be discovered using just oracle comparison queries. Here we compute it directly. This measurement can be computed for each parameterization. As described in the paper, we need $ ng \choose 2 $ equations because there are $ng \choose 2$ $B$ matrices. Generally speaking, all unique paramaterizations as described above would create enough measurements.

**Exceptions: when the number of groups is 2 or 4, we need to use parameterizations that are not just 2-tuples.** Here is why:

In the ng = 2 case, we pick one group to be s and the other to be the trivial rate. We don't have enough groups to run the described procedure.
    
In the ng = 4 case, consider $<s, s, o, o>$. Note that the B vectors associated
with this are $B_{02}$, $B_{03}$, $B_{12}$, $B_{13}$ (all s,o pairs). Now consider $<o, o, s, s>$. The
B vectors are the exact same! So in the ng = 4 case, enumerating all $4 \choose 2$ ways to place $s$ creates duplicate uses of the B vectors. We get around this by also using some 1-tuple paramaterizations. This means just one group is chosen as $s$ and the rest are set to the trivial rate. Note that this is not a problem for any other number of groups.

After getting *b_measurements*, we use some linear algebra to solve the system of equations (rather system of matrices). This is implemented in the *FairParametizer* class in *fpme_utils.py*. The most important function is *recover_B_lambda_from_measurements*.

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import numpy as np
from sklearn.datasets import make_spd_matrix

import sys
sys.path.append('../')
from common import normalize, Sphere, Oracle
from fpme_utils import FairParametizer, create_a_B_lamb_T, compute_B_err

In [3]:
np.random.seed(7)
ng = 3
nc = 3

q = nc ** 2 - nc
v = ng * (ng - 1) // 2

In [4]:
a, B, lamb, T = create_a_B_lamb_T(ng, nc, q)

In [5]:
a

array([0.04754303, 0.48591967, 0.27314596, 0.45074688, 0.60932541,
       0.33550382])

In [6]:
fp = FairParametizer(q, ng)

In [7]:
params = fp.create_paramaterization()
params

[(0, 1), (0, 2), (1, 2)]

In [8]:
# this would determined by the FPME algorithm. Here we use the theoretical
# expected, which is lamb / (1 - lamb) * \sum of some B_ij's
b_measurements = [np.matrix(np.zeros((q,q), dtype=np.float64)) for _ in range(v)]
for i, p in enumerate(params):
    for k in range(ng):
        if k != p.i and k != p.j:
            # if p.i or p.j is -1, that means the parameterization is
            # not of type (m C 2), where 2 vectors are s and the rest trivial
            if p.i == -1:
                b_measurements[i] += B[k][p.j] * lamb / (1. - lamb)
            elif p.j == -1:
                b_measurements[i] += B[k][p.i] * lamb / (1. - lamb)
            else:
                b_measurements[i] += (B[k][p.i] + B[k][p.j]) * lamb / (1. - lamb)
                

In [9]:
B_hat, lamb_hat = fp.recover_B_lambda_from_measurements(b_measurements, params)

In [10]:
lamb, lamb_hat

(0.6096160708461682, 0.6096160708461681)

In [11]:
print("lambda amount off:", abs(lamb_hat - lamb))
print("B total err:", compute_B_err(B_hat, B))

lambda amount off: 1.1102230246251565e-16
B total err: 5.544489575524005e-16
